# Memory Systems Implementation - Lesson 14

**Tutorial 18: Interactive Implementation of Memory Patterns**

This notebook demonstrates:
- Working memory management (trimming & summarization)
- Long-term memory with Chroma vector database
- Search-o1 pattern with Reason-in-Documents
- Context engineering techniques (MMR, compression, ordering)
- Cost/latency optimization strategies

**Prerequisites:**
- Read `memory_systems_fundamentals.md`
- Read `context_engineering_guide.md`
- Familiarity with RAG basics (`04_Agentic_RAG.md`)

**Execution Modes:**
- **DEMO**: Quick demonstration (~8-10 minutes, 10 queries, 100 documents)
- **FULL**: Comprehensive analysis (~30-40 minutes, 50 queries, 500 documents)

---

## Setup & Configuration

### ⚠️ Cost Warning

**DEMO mode**: No API costs (uses mock LLM calls)  
**FULL mode**: Estimated API costs if LLM enabled:
- ~$0.50 - $1.00 for GPT-4 calls
- ~$0.10 - $0.25 for GPT-3.5-turbo calls

Set `EXECUTION_MODE = 'DEMO'` to avoid costs.

In [ ]:
# Configuration Cell
import sys
from pathlib import Path

# Add lesson-14 to path for imports
sys.path.insert(0, str(Path.cwd()))

# Execution mode
EXECUTION_MODE = 'DEMO'  # Change to 'FULL' for comprehensive analysis

# Optional: Enable real LLM calls (requires API key)
USE_LLM = False  # Set to True to use actual LLM API calls

# Mode-specific configs
if EXECUTION_MODE == 'DEMO':
    NUM_QUERIES = 10
    NUM_DOCUMENTS = 100
    CHUNK_SIZE = 500
elif EXECUTION_MODE == 'FULL':
    NUM_QUERIES = 50
    NUM_DOCUMENTS = 500
    CHUNK_SIZE = 1000
else:
    raise ValueError(f"EXECUTION_MODE must be 'DEMO' or 'FULL', got '{EXECUTION_MODE}'")

print(f"✅ Execution Mode: {EXECUTION_MODE}")
print(f"   - Queries: {NUM_QUERIES}")
print(f"   - Documents: {NUM_DOCUMENTS}")
print(f"   - LLM Enabled: {USE_LLM}")
print(f"   - Estimated Runtime: {'<10 min' if EXECUTION_MODE == 'DEMO' else '30-40 min'}")

In [ ]:
# Import dependencies
import json
import time
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from typing import Any

# ChromaDB for vector storage
import chromadb
from chromadb.config import Settings

# Our helper functions (tested in tests/test_memory_systems_notebook.py)
from memory_systems_helpers import (
    validate_execution_mode,
    calculate_mmr_score,
    select_documents_mmr,
    count_tokens,
    trim_conversation_history,
    simulate_summarization,
    calculate_search_o1_overhead,
    calculate_compression_roi,
    export_results_json,
)

# Validate execution mode
validate_execution_mode(EXECUTION_MODE, raise_on_invalid=True)

print("✅ All imports successful")

## Part 1: Working Memory Management

Demonstrates short-term memory techniques:
1. Token counting and budget tracking
2. Conversation trimming (FIFO strategy)
3. Conversation summarization
4. Cost comparison: trimming vs summarization

In [ ]:
# Sample conversation history
conversation_history = [
    {"role": "user", "content": "What is the Bhagavad Gita about?"},
    {"role": "assistant", "content": "The Bhagavad Gita is a 700-verse Hindu scripture that is part of the epic Mahabharata. It consists of a dialogue between Prince Arjuna and the god Krishna on the battlefield of Kurukshetra."},
    {"role": "user", "content": "Who is Arjuna?"},
    {"role": "assistant", "content": "Arjuna is one of the Pandava brothers, a skilled archer and warrior. In the Gita, he is faced with moral dilemmas about fighting his own relatives in battle."},
    {"role": "user", "content": "What is dharma?"},
    {"role": "assistant", "content": "Dharma is a central concept meaning duty, righteousness, or moral order. Krishna advises Arjuna to follow his dharma as a warrior, even in difficult circumstances."},
    {"role": "user", "content": "Tell me about karma yoga."},
    {"role": "assistant", "content": "Karma Yoga is the path of selfless action. The Gita teaches performing one's duty without attachment to results, dedicating actions to the divine rather than personal gain."},
]

# Count tokens in conversation
total_tokens = sum(count_tokens(msg["content"]) for msg in conversation_history)
print(f"\n📊 Original Conversation:")
print(f"   - Messages: {len(conversation_history)}")
print(f"   - Total Tokens: {total_tokens}")
print(f"   - Cost (@$0.03/1K tokens): ${(total_tokens / 1000) * 0.03:.4f}")

In [ ]:
# Strategy 1: Trimming (FIFO - keep most recent)
max_tokens_budget = 100
trimmed_history = trim_conversation_history(conversation_history, max_tokens_budget)

trimmed_tokens = sum(count_tokens(msg["content"]) for msg in trimmed_history)
print(f"\n✂️ After Trimming (max {max_tokens_budget} tokens):")
print(f"   - Messages Kept: {len(trimmed_history)} / {len(conversation_history)}")
print(f"   - Tokens: {trimmed_tokens}")
print(f"   - Reduction: {((total_tokens - trimmed_tokens) / total_tokens) * 100:.1f}%")
print(f"   - Cost: ${(trimmed_tokens / 1000) * 0.03:.4f}")
print(f"\n   Kept messages (most recent):")
for msg in trimmed_history:
    print(f"     - {msg['role']}: {msg['content'][:50]}...")

In [ ]:
# Strategy 2: Summarization
compression_ratio = 0.3  # Target 30% of original tokens
summary_msg, orig_tokens, summary_tokens = simulate_summarization(
    conversation_history, compression_ratio
)

print(f"\n📝 After Summarization (ratio={compression_ratio}):")
print(f"   - Original Tokens: {orig_tokens}")
print(f"   - Summary Tokens: {summary_tokens}")
print(f"   - Compression: {(summary_tokens / orig_tokens) * 100:.1f}% of original")
print(f"   - Cost: ${(summary_tokens / 1000) * 0.03:.4f}")
print(f"\n   Summary: {summary_msg['content'][:100]}...")

In [ ]:
# Visualization: Token usage comparison
strategies = ['Original', 'Trimming', 'Summarization']
token_counts = [total_tokens, trimmed_tokens, summary_tokens]
costs = [(t / 1000) * 0.03 for t in token_counts]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

# Tokens comparison
ax1.bar(strategies, token_counts, color=['#1f77b4', '#ff7f0e', '#2ca02c'])
ax1.set_ylabel('Tokens')
ax1.set_title('Token Usage by Strategy')
ax1.axhline(y=max_tokens_budget, color='r', linestyle='--', label=f'Budget ({max_tokens_budget})')
ax1.legend()

# Cost comparison
ax2.bar(strategies, costs, color=['#1f77b4', '#ff7f0e', '#2ca02c'])
ax2.set_ylabel('Cost ($)')
ax2.set_title('API Cost by Strategy (@$0.03/1K tokens)')

plt.tight_layout()
plt.show()

print(f"\n💡 Insight: Summarization achieves {((total_tokens - summary_tokens) / total_tokens) * 100:.0f}% token reduction")
print(f"   vs {((total_tokens - trimmed_tokens) / total_tokens) * 100:.0f}% for trimming, but preserves older context.")

## Part 2: Long-Term Memory with Chroma

Set up persistent vector database for episodic/semantic memory.

In [ ]:
# Initialize Chroma persistent client
chroma_path = Path.cwd() / "data" / "chroma_memory_demo"
chroma_path.mkdir(parents=True, exist_ok=True)

client = chromadb.PersistentClient(
    path=str(chroma_path),
    settings=Settings(
        anonymized_telemetry=False,
        allow_reset=True,
    )
)

# Create/get collection
collection_name = "lesson14_memory_demo"
try:
    client.delete_collection(collection_name)  # Clean slate
except:
    pass

collection = client.create_collection(
    name=collection_name,
    metadata={"description": "Memory systems tutorial demo collection"}
)

print(f"✅ Chroma client initialized at {chroma_path}")
print(f"   Collection: {collection_name}")

In [ ]:
# Ingest sample data from lesson-14
# Read multi_agent_fundamentals.md and chunk it
source_file = Path.cwd() / "multi_agent_fundamentals.md"

if source_file.exists():
    with open(source_file) as f:
        content = f.read()
    
    # Simple chunking by paragraphs
    paragraphs = [p.strip() for p in content.split('\n\n') if len(p.strip()) > 50]
    
    # Limit to NUM_DOCUMENTS
    chunks = paragraphs[:NUM_DOCUMENTS]
    
    # Create mock embeddings (in real use, use sentence-transformers or OpenAI)
    # For demo, use random embeddings with some semantic structure
    np.random.seed(42)
    embeddings = np.random.rand(len(chunks), 384).tolist()  # 384-dim like sentence-transformers
    
    # Add to collection with metadata
    collection.add(
        documents=chunks,
        embeddings=embeddings,
        metadatas=[{"source": "multi_agent_fundamentals.md", "chunk_id": i} for i in range(len(chunks))],
        ids=[f"doc_{i}" for i in range(len(chunks))]
    )
    
    print(f"✅ Ingested {len(chunks)} chunks into Chroma")
    print(f"   Source: multi_agent_fundamentals.md")
    print(f"   Embedding Dimension: 384")
else:
    print(f"⚠️ Source file not found: {source_file}")
    print(f"   Using synthetic data instead...")
    
    # Fallback: create synthetic chunks
    synthetic_chunks = [
        f"This is synthetic document {i} about multi-agent systems and memory patterns."
        for i in range(NUM_DOCUMENTS)
    ]
    np.random.seed(42)
    embeddings = np.random.rand(len(synthetic_chunks), 384).tolist()
    
    collection.add(
        documents=synthetic_chunks,
        embeddings=embeddings,
        metadatas=[{"source": "synthetic", "chunk_id": i} for i in range(len(synthetic_chunks))],
        ids=[f"doc_{i}" for i in range(len(synthetic_chunks))]
    )
    print(f"✅ Ingested {len(synthetic_chunks)} synthetic chunks")

In [ ]:
# Basic retrieval example
query_text = "What are the key components of multi-agent systems?"
np.random.seed(123)
query_embedding = np.random.rand(384).tolist()

results = collection.query(
    query_embeddings=[query_embedding],
    n_results=5
)

print(f"\n🔍 Basic Retrieval Example:")
print(f"   Query: {query_text}")
print(f"   Top 5 Results:")
for i, (doc, meta) in enumerate(zip(results['documents'][0], results['metadatas'][0])):
    print(f"\n   {i+1}. (Source: {meta['source']}, Chunk {meta['chunk_id']})")
    print(f"      {doc[:100]}...")

## Part 3: Context Engineering - MMR Selection

Compare different λ values for relevance vs diversity trade-off.

In [ ]:
# Exercise 1: MMR with different lambda values
# Retrieve more documents than needed, then select with MMR
candidate_results = collection.query(
    query_embeddings=[query_embedding],
    n_results=20  # Over-retrieve
)

candidate_embeddings = np.array(candidate_results['embeddings'][0])
k = 5  # Select top 5 with MMR

lambda_values = [0.0, 0.3, 0.5, 0.7, 1.0]
mmr_results = {}

for lambda_val in lambda_values:
    selected_indices = select_documents_mmr(
        query_embedding,
        candidate_embeddings.tolist(),
        k=k,
        lambda_param=lambda_val
    )
    mmr_results[lambda_val] = selected_indices

print(f"\n🎯 MMR Selection Results (k={k} from 20 candidates):")
print(f"\nλ | Selected Document Indices")
print("-" * 40)
for lambda_val, indices in mmr_results.items():
    print(f"{lambda_val:.1f} | {indices}")

print(f"\n💡 Interpretation:")
print(f"   λ=0.0 (pure relevance): Selects most similar documents (may be redundant)")
print(f"   λ=1.0 (pure diversity): Maximizes diversity (may sacrifice relevance)")
print(f"   λ=0.3-0.7 (balanced): Good trade-off for most use cases")

In [ ]:
# Visualization: Compare overlap between λ=0 and λ=0.7
lambda_0_set = set(mmr_results[0.0])
lambda_07_set = set(mmr_results[0.7])

overlap = len(lambda_0_set & lambda_07_set)
unique_to_0 = len(lambda_0_set - lambda_07_set)
unique_to_07 = len(lambda_07_set - lambda_0_set)

fig, ax = plt.subplots(figsize=(8, 5))
categories = ['Overlap', 'Unique to λ=0.0\n(pure relevance)', 'Unique to λ=0.7\n(balanced)']
counts = [overlap, unique_to_0, unique_to_07]
colors = ['#9467bd', '#1f77b4', '#2ca02c']

ax.bar(categories, counts, color=colors)
ax.set_ylabel('Number of Documents')
ax.set_title('MMR Selection Comparison: λ=0.0 vs λ=0.7')
ax.set_ylim(0, k)

plt.tight_layout()
plt.show()

print(f"\n📊 Diversity Impact:")
print(f"   Overlap: {overlap}/{k} documents")
print(f"   Diversity brings in {unique_to_07} new documents vs pure relevance")

## Part 4: Compression ROI Calculator

Exercise 2: Calculate cost savings from context compression.

In [ ]:
# Scenario: 5 retrieved documents, each ~500 tokens
original_doc_tokens = 500
num_docs = 5
original_total = original_doc_tokens * num_docs

# Test different compression strategies
compression_strategies = {
    "No compression": 1.0,
    "Light (remove duplicates)": 0.8,
    "Medium (extractive summary)": 0.5,
    "Aggressive (LLMLingua)": 0.2,
}

cost_per_1k = 0.03  # GPT-4 input pricing

print(f"\n💰 Compression ROI Analysis:")
print(f"   Original: {num_docs} docs × {original_doc_tokens} tokens = {original_total} tokens")
print(f"   Pricing: ${cost_per_1k}/1K tokens\n")
print(f"Strategy                    | Tokens  | Cost      | Savings   | Ratio")
print("-" * 75)

roi_data = []
for strategy, ratio in compression_strategies.items():
    compressed_tokens = int(original_total * ratio)
    roi = calculate_compression_roi(original_total, compressed_tokens, cost_per_1k)
    
    roi_data.append(roi)
    print(f"{strategy:27} | {compressed_tokens:6} | ${roi['compressed_cost']:.4f} | ${roi['cost_savings']:.4f}  | {ratio:.1f}x")

print(f"\n💡 Real-world Example (from COMPASS_ARTIFACT_ANALYSIS.md):")
print(f"   Context compression: $24 → $12 → $4.80 (5x ROI)")
print(f"   Strategy: Deduplication + extractive summarization")

In [ ]:
# Visualization: Cost savings
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Cost comparison
strategies_list = list(compression_strategies.keys())
costs = [roi['compressed_cost'] for roi in roi_data]
ax1.barh(strategies_list, costs, color='#ff7f0e')
ax1.axvline(x=roi_data[0]['original_cost'], color='r', linestyle='--', 
            label=f"Original (${roi_data[0]['original_cost']:.4f})")
ax1.set_xlabel('Cost ($)')
ax1.set_title('API Cost by Compression Strategy')
ax1.legend()
ax1.invert_yaxis()

# Savings
savings = [roi['cost_savings'] for roi in roi_data]
ax2.barh(strategies_list, savings, color='#2ca02c')
ax2.set_xlabel('Savings ($)')
ax2.set_title('Cost Savings vs Original')
ax2.invert_yaxis()

plt.tight_layout()
plt.show()

## Part 5: Search-o1 Pattern Implementation

Demonstrate the Search-o1 reasoning pattern with Reason-in-Documents.

In [ ]:
# Search-o1 Pattern: Multi-branch search with reasoning
def simulate_search_o1(query: str, num_branches: int = 3) -> dict[str, Any]:
    """Simulate Search-o1 pattern with token tracking."""
    
    # Step 1: Generate search queries for multiple branches
    search_queries = [
        f"Branch {i+1}: {query} (aspect {i+1})" for i in range(num_branches)
    ]
    search_tokens = sum(count_tokens(q) for q in search_queries)
    
    # Step 2: Retrieve documents for each branch
    np.random.seed(42)
    retrieval_tokens = 0
    all_documents = []
    
    for search_query in search_queries:
        # Mock retrieval (in real use, query vector DB)
        query_emb = np.random.rand(384).tolist()
        results = collection.query(
            query_embeddings=[query_emb],
            n_results=3
        )
        docs = results['documents'][0]
        all_documents.extend(docs)
        retrieval_tokens += sum(count_tokens(doc) for doc in docs)
    
    # Step 3: Reason-in-Documents (condensation)
    # Simulate LLM reasoning over all retrieved docs
    condensation_input = f"Reasoning over {len(all_documents)} documents for query: {query}"
    condensation_input_tokens = count_tokens(condensation_input) + retrieval_tokens
    
    # Condensation reduces to ~30% of input
    condensation_output_tokens = int(condensation_input_tokens * 0.3)
    condensation_tokens = condensation_input_tokens + condensation_output_tokens
    
    # Step 4: Calculate overhead
    baseline_tokens = count_tokens(query) + retrieval_tokens  # Without Search-o1
    overhead = calculate_search_o1_overhead(
        search_tokens, retrieval_tokens, condensation_tokens, baseline_tokens
    )
    
    return {
        "num_branches": num_branches,
        "search_tokens": search_tokens,
        "retrieval_tokens": retrieval_tokens,
        "condensation_tokens": condensation_tokens,
        "total_overhead": overhead["total_overhead"],
        "overhead_percentage": overhead.get("overhead_percentage", 0),
        "documents_retrieved": len(all_documents),
    }

# Run Search-o1 simulation
test_query = "Explain multi-agent memory coordination patterns"
search_o1_result = simulate_search_o1(test_query, num_branches=3)

print(f"\n🔬 Search-o1 Pattern Simulation:")
print(f"   Query: {test_query}")
print(f"   Branches: {search_o1_result['num_branches']}")
print(f"\n   Token Breakdown:")
print(f"     - Search queries: {search_o1_result['search_tokens']}")
print(f"     - Retrieved docs: {search_o1_result['retrieval_tokens']}")
print(f"     - Condensation: {search_o1_result['condensation_tokens']}")
print(f"     - Total overhead: {search_o1_result['total_overhead']}")
print(f"     - Overhead %: {search_o1_result['overhead_percentage']:.1f}%")
print(f"\n   Documents retrieved: {search_o1_result['documents_retrieved']}")

In [ ]:
# Compare overhead for different branch counts
branch_counts = [1, 2, 3, 4, 5]
overhead_comparison = []

for num_branches in branch_counts:
    result = simulate_search_o1(test_query, num_branches=num_branches)
    overhead_comparison.append(result)

# Visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Token breakdown by branch count
search_tokens_list = [r['search_tokens'] for r in overhead_comparison]
retrieval_tokens_list = [r['retrieval_tokens'] for r in overhead_comparison]
condensation_tokens_list = [r['condensation_tokens'] for r in overhead_comparison]

x = np.arange(len(branch_counts))
width = 0.25

ax1.bar(x - width, search_tokens_list, width, label='Search', color='#1f77b4')
ax1.bar(x, retrieval_tokens_list, width, label='Retrieval', color='#ff7f0e')
ax1.bar(x + width, condensation_tokens_list, width, label='Condensation', color='#2ca02c')

ax1.set_xlabel('Number of Branches')
ax1.set_ylabel('Tokens')
ax1.set_title('Search-o1 Token Breakdown by Branch Count')
ax1.set_xticks(x)
ax1.set_xticklabels(branch_counts)
ax1.legend()

# Overhead percentage
overhead_pcts = [r['overhead_percentage'] for r in overhead_comparison]
ax2.plot(branch_counts, overhead_pcts, marker='o', linewidth=2, markersize=8, color='#d62728')
ax2.set_xlabel('Number of Branches')
ax2.set_ylabel('Overhead (%)')
ax2.set_title('Search-o1 Overhead vs Baseline')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n💡 Trade-off Analysis:")
print(f"   More branches → More comprehensive search")
print(f"   But: Overhead increases linearly with branches")
print(f"   Recommendation: 2-3 branches for most use cases")

## Part 6: Multi-Agent Memory Coordination

Exercise 3: Demonstrate shared vs private memory in multi-agent systems.

In [ ]:
# Scenario: 3 agents with different memory configs
agents = {
    "Agent A (Shared Only)": {
        "shared_memory": True,
        "private_memory": False,
        "memory_size_tokens": 0,  # Only uses shared
    },
    "Agent B (Private Only)": {
        "shared_memory": False,
        "private_memory": True,
        "memory_size_tokens": 500,
    },
    "Agent C (Hybrid)": {
        "shared_memory": True,
        "private_memory": True,
        "memory_size_tokens": 300,
    },
}

# Shared memory (accessible to all agents)
shared_memory_tokens = 1000

# Calculate total memory footprint
print(f"\n🤝 Multi-Agent Memory Coordination:")
print(f"\n   Shared Memory: {shared_memory_tokens} tokens (accessible to all)\n")

for agent_name, config in agents.items():
    total_access = 0
    if config["shared_memory"]:
        total_access += shared_memory_tokens
    if config["private_memory"]:
        total_access += config["memory_size_tokens"]
    
    print(f"   {agent_name}:")
    print(f"     - Shared: {'✓' if config['shared_memory'] else '✗'}")
    print(f"     - Private: {'✓' if config['private_memory'] else '✗'} ({config['memory_size_tokens']} tokens)")
    print(f"     - Total Access: {total_access} tokens\n")

# Calculate system-wide memory cost
total_private = sum(a["memory_size_tokens"] for a in agents.values())
total_system = shared_memory_tokens + total_private
cost_per_1k = 0.03
total_cost = (total_system / 1000) * cost_per_1k

print(f"   System-Wide Memory:")
print(f"     - Shared: {shared_memory_tokens} tokens")
print(f"     - Private (total): {total_private} tokens")
print(f"     - Total: {total_system} tokens")
print(f"     - Cost: ${total_cost:.4f} per query")

print(f"\n💡 Design Principle:")
print(f"   Use shared memory for common knowledge (ontologies, facts)")
print(f"   Use private memory for agent-specific state/history")
print(f"   Hybrid approach balances coordination and specialization")

## Part 7: Results Export & Metrics

Generate summary statistics and export to JSON for dashboard.

In [ ]:
# Aggregate metrics from all exercises
summary_statistics = {
    "trimming_reduction": {
        "mean": ((total_tokens - trimmed_tokens) / total_tokens),
        "std": 0.05,  # Mock std
    },
    "summarization_reduction": {
        "mean": ((total_tokens - summary_tokens) / total_tokens),
        "std": 0.08,
    },
    "mmr_diversity_impact": {
        "mean": unique_to_07 / k,
        "std": 0.12,
    },
    "compression_roi": {
        "mean": np.mean([r["cost_savings"] for r in roi_data[1:]]),  # Exclude no compression
        "std": np.std([r["cost_savings"] for r in roi_data[1:]]),
    },
    "search_o1_overhead_pct": {
        "mean": np.mean(overhead_pcts),
        "std": np.std(overhead_pcts),
    },
}

# Radar chart data
radar_chart_data = {
    "labels": list(summary_statistics.keys()),
    "values": [summary_statistics[k]["mean"] for k in summary_statistics.keys()],
}

# Detailed results (per-exercise breakdown)
detailed_results = [
    {
        "exercise": "Working Memory - Trimming",
        "metric": "token_reduction",
        "value": summary_statistics["trimming_reduction"]["mean"],
    },
    {
        "exercise": "Working Memory - Summarization",
        "metric": "token_reduction",
        "value": summary_statistics["summarization_reduction"]["mean"],
    },
    {
        "exercise": "Context Engineering - MMR",
        "metric": "diversity_impact",
        "value": summary_statistics["mmr_diversity_impact"]["mean"],
    },
    {
        "exercise": "Compression ROI",
        "metric": "cost_savings_usd",
        "value": summary_statistics["compression_roi"]["mean"],
    },
    {
        "exercise": "Search-o1 Pattern",
        "metric": "overhead_percentage",
        "value": summary_statistics["search_o1_overhead_pct"]["mean"],
    },
]

print(f"\n📊 Summary Statistics:")
for metric_name, stats in summary_statistics.items():
    print(f"   {metric_name}: {stats['mean']:.3f} (±{stats['std']:.3f})")

In [ ]:
# Export results to JSON
output_path = Path.cwd() / "results" / "memory_systems_demo_results.json"
output_path.parent.mkdir(parents=True, exist_ok=True)

export_results_json(
    output_path=output_path,
    summary_statistics=summary_statistics,
    radar_chart_data=radar_chart_data,
    detailed_results=detailed_results,
    execution_mode=EXECUTION_MODE,
)

print(f"\n✅ Results exported to: {output_path}")
print(f"   - Execution Mode: {EXECUTION_MODE}")
print(f"   - Metrics: {len(summary_statistics)}")
print(f"   - Detailed Results: {len(detailed_results)} exercises")
print(f"\n💡 Load this JSON in lesson-9-11/evaluation_dashboard.py to visualize!")

## Summary & Key Takeaways

### What We Demonstrated:

1. **Working Memory Management**
   - Trimming (FIFO): Simple, preserves recent context
   - Summarization: Better compression, preserves older context
   - Trade-off: Cost vs completeness

2. **Long-Term Memory (Chroma)**
   - Persistent vector storage
   - Metadata-filtered retrieval
   - Local deployment (no external API costs)

3. **Context Engineering**
   - **MMR Selection**: λ parameter balances relevance vs diversity
   - **Compression**: 5x cost savings with aggressive strategies
   - **ROI Math**: Quantify cost/latency benefits

4. **Search-o1 Pattern**
   - Multi-branch search for comprehensive coverage
   - Reason-in-Documents condensation
   - Overhead grows linearly with branches (2-3 optimal)

5. **Multi-Agent Memory**
   - Shared memory: Common knowledge, reduces duplication
   - Private memory: Agent-specific state
   - Hybrid: Best of both worlds

### Next Steps:

1. **Explore Diagrams:**
   - `diagrams/memory_types_taxonomy.png` - Memory type hierarchy
   - `diagrams/context_engineering_workflow.png` - Selection → Compression → Ordering pipeline
   - `diagrams/search_o1_architecture.png` - Branching reasoning workflow

2. **Read Companion Tutorials:**
   - `memory_systems_fundamentals.md` - Theory and patterns
   - `context_engineering_guide.md` - Deep dive into optimization

3. **Apply to Your Projects:**
   - Start with DEMO mode to prototype
   - Use FULL mode for production evaluation
   - Track costs with compression ROI calculator

4. **Experiment:**
   - Try different λ values for your use case
   - Test compression strategies with real data
   - Benchmark Search-o1 vs standard RAG

---

**🎓 Tutorial Complete!**

You've successfully implemented and evaluated 5 memory system patterns. Check the exported JSON for detailed metrics, and visualize in the evaluation dashboard.

**Estimated Reading Time:** 45-60 minutes (with code execution)  
**Execution Time:** ~{runtime} minutes ({EXECUTION_MODE} mode)

🤖 Generated with [Claude Code](https://claude.com/claude-code)